In [ ]:
#Import csv final Step 4 results

import pandas as pd
import io
import requests

# URL
url = "https://github.com/NataKrj/AI-project-2024/raw/main/Results%2026.12.2024/Step_4.1_company_risk_scores.csv"
s = requests.get(url).content
# For the delimiter and error handling
c = pd.read_csv(io.StringIO(s.decode('utf-8')), sep=',', on_bad_lines='skip', engine='python')


print(c.head())

          company_name Sanctioned Names          Status      matched_keywords  \
0                   9g         No Match           error  No relevant keywords   
1  a.b.c. kickers bvba         No Match  ENT LP Stopped  No relevant keywords   
2       a.d. froidmont         No Match      EU Stopped                 court   
3        a.f. security         No Match      EU Stopped                   NaN   
4        a.f.m. new sa         No Match           error                   NaN   

   Total_Score risk_level  
0         12.0       high  
1         10.0       high  
2         15.0       high  
3          5.0     medium  
4          2.0     medium  


In [ ]:
!pip install pymongo
!pip install dnspython

In [ ]:
from pymongo import MongoClient
import json
from io import StringIO

# MongoDB connection setup
uri = "mongodb+srv://xxxxxxxxxxxxxxx.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"  # Replace with your MongoDB URI
client = MongoClient(uri)

# Test the connection
try:
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")
except Exception as e:
    print(f"Error connecting to MongoDB: {e}")
    exit()

# Access the database and collection
db_name = "Company_scoring"
collection_name = "Company_risk_scores"
db = client[db_name]
collection = db[collection_name]

# Path to the uploaded CSV file
file_path = 'Step_4.1_company_risk_scores.csv'

try:
    # Load the CSV content into a DataFrame
    df = pd.read_csv(
        file_path,
        sep=',',             # delimiter
        quotechar='"',
        on_bad_lines='skip', # Skip problematic rows
        engine='python'      # Use Python engine--> adjusted
    )

    # Check  DataFrame
    print("Preview of the imported DataFrame:")
    print(df.head())

    # Check for rows with mismatched columns
    print("\nNumber of columns in DataFrame:", len(df.columns))
    print("Column Names:", df.columns.tolist())

    # Validate rows with missing/ extra fields
    if df.isnull().any().any():
        print("\nWarning: Missing values detected in the following rows:")
        print(df[df.isnull().any(axis=1)])

        # Fill missing values with "N/A"
        df.fillna("N/A", inplace=True)
        print("\nData after filling missing values:")
        print(df.head())
    else:
        print("\nNo missing values detected.")

    # Rename columns to match desired fields if necessary
    df.columns = ["company_name", "Sanctioned Names", "Status", "matched_keywords", "Total_Score", "risk_level"]

    # Convert DataFrame to a list of JSON objects
    data = json.loads(df.to_json(orient='records'))

    # Preview the data to be inserted
    print("Preview of data to be inserted into MongoDB:")
    print(json.dumps(data[:5], indent=4))

    # Insert data into MongoDB
    if data:
        collection.insert_many(data)
        print("Data successfully imported into MongoDB!")
    else:
        print("No data found in the CSV file.")
except Exception as e:
    print(f"An error occurred: {e}")


Successfully connected to MongoDB!
Preview of the imported DataFrame:
          company_name Sanctioned Names          Status      matched_keywords  \
0                   9g         No Match           error  No relevant keywords   
1  a.b.c. kickers bvba         No Match  ENT LP Stopped  No relevant keywords   
2       a.d. froidmont         No Match      EU Stopped                 court   
3        a.f. security         No Match      EU Stopped                   NaN   
4        a.f.m. new sa         No Match           error                   NaN   

   Total_Score risk_level  
0         12.0       high  
1         10.0       high  
2         15.0       high  
3          5.0     medium  
4          2.0     medium  

Number of columns in DataFrame: 6
Column Names: ['company_name', 'Sanctioned Names', 'Status', 'matched_keywords', 'Total_Score', 'risk_level']

                  company_name Sanctioned Names                       Status  \
3                a.f. security         No Match  

#Vizualization in the Mongo database link: https://charts.mongodb.com/charts-mongodb-pbuqwjz/public/dashboards/673dbfd4-c3c0-45fb-818c-bec4f2336c73